# Environment Development ("BabyAI-Text")

This notebook builds the BabyAI ACT-PRM environment, inspects the real state, renders the prompt with tools, and shows the gold trajectory. This is a compact setup similar to the exmaple notebook.



In [2]:
import os

os.environ["HF_HOME"] = os.path.expanduser("~/.cache/huggingface")
os.environ["TRANSFORMERS_CACHE"] = os.path.expanduser("~/.cache/huggingface/transformers")

In [3]:
from omegaconf import OmegaConf
from rich import print as rich_print

from transformers import AutoTokenizer

# Get a tokenizer
model_config = OmegaConf.load("../configs/model/hf_qwen3_4b_inst_2507.yaml")
# Override cache_dir to a writable path (config defaults to /scr/...)
model_config.model_config["cache_dir"] = os.path.expanduser("~/.cache/huggingface/models")

hf_tokenizer = AutoTokenizer.from_pretrained(**model_config.model_config)

/Users/madisonho/Documents/act-prm-tinker/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/madisonho/Documents/act-prm-tinker/.venv/lib/python3.12/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
def rich_print_messages(
    msg_text: str,
    bos_token: str = "<|im_start|>",
    eos_token: str = "<|im_end|>\n",
    tool_call_bos_token: str = "<tool_call>",
    tool_call_eos_token: str = "</tool_call>",
    tool_response_bos_token: str = "<tool_response>",
    tool_response_eos_token: str = "</tool_response>",
):
    # Split into messages
    messages = msg_text.split(eos_token)

    system_bos = f"{bos_token}system"
    user_bos = f"{bos_token}user"
    assistant_bos = f"{bos_token}assistant"

    for ix, message in enumerate(messages):
        # system prompt
        if message.startswith(system_bos):
            messages[ix] = f"[bright_yellow]{message}[/bright_yellow]"
        # user messages
        elif message.startswith(user_bos):
            messages[ix] = f"[bright_red]{message}[/bright_red]"
        # assistant messages
        elif message.startswith(assistant_bos):
            messages[ix] = f"[bright_green]{message}[/bright_green]"

        # tool calls
        if tool_call_bos_token in messages[ix] and tool_call_eos_token in messages[ix]:
            messages[ix] = messages[ix].replace(tool_call_bos_token, f"[bright_cyan]{tool_call_bos_token}")
            messages[ix] = messages[ix].replace(tool_call_eos_token, f"{tool_call_eos_token}[/bright_cyan]")
        # tool responses
        if tool_response_bos_token in messages[ix] and tool_response_eos_token in messages[ix]:
            messages[ix] = messages[ix].replace(tool_response_bos_token, f"[bright_magenta]{tool_response_bos_token}")
            messages[ix] = messages[ix].replace(tool_response_eos_token, f"{tool_response_eos_token}[/bright_magenta]")

    msgs_text = eos_token.join(messages)
    try:
        rich_print(msgs_text)
    except Exception:
        print(msgs_text)

## BabyAI-Text (ACT-PRM)

We now build the BabyAI environment and inspect the real state plus the tool schema.


In [5]:
from act_prm.environments import get_env

babyai_env = get_env(
    name="babyai_text",
    env_name="BabyAI-MixedTestLocal-v0",
    num_train_samples=3,
    num_val_samples=1,
    num_test_samples=1,
    max_turns=20,
    seed=0,
)

babyai_state = babyai_env.reset()

babyai_state.system_prompt, babyai_state.new_messages[0]["content"][:400]

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/Users/madisonho/Documents/Grounding_LLMs_with_online_RL/babyai-text/gym-minigrid/gym_minigrid/minigrid.py:601: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  mask = np.zeros(shape=(grid.width, grid.height), dtype=np.bool)


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
system_message = {"role": "system", "content": babyai_state.system_prompt}

messages = hf_tokenizer.apply_chat_template(
    [system_message] + babyai_state.new_messages,
    tokenize=False,
    tools=babyai_state.tools,
)
rich_print_messages(messages)

<|im_start|>system
You are a helpful assistant that can answer questions and call tools.

You are an agent playing BabyAI-Text, a grid-based game with text observations.
Your goal is to follow the mission and reach the win condition.

## Rules:
- You must call exactly one tool per turn to choose an action.
- Only the listed tools are valid actions.
- You have 20 steps to complete the task. Restarting is forbidden.

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "name": "turn_left", "description": "Rotate the agent 90 degrees to the left.", "parameters": 
{"type": "object", "properties": {}}}
{"type": "function", "name": "turn_right", "description": "Rotate the agent 90 degrees to the right.", 
"parameters": {"type": "object", "properties": {}}}
{"type": "function", "name": "go_forward", "description": "Move forward one cell.", "parameters": {"type": 
"object", "properties": {}}}
{"type": "function", "name": "pick_up", "description": "Pick up the object in front of the agent.", "parameters": 
{"type": "object", "properties": {}}}
{"type": "function", "name": "drop", "description": "Drop the object the agent is carrying.", "parameters": 
{"type": "object", "properties": {}}}
{"type": "function", "name": "toggle", "description": "Toggle (use) the object in front of the agent.", 
"parameters": {"type": "object", "properties": {}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML 
tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call><|im_end|>
<|im_start|>user
You are playing 'BabyAI-Text'.
Your goal is to open the red door.
Available actions are turn left, turn right, go forward, pick up, drop, toggle.
You see a wall 4 steps forward. You see a yellow key 2 steps left and 1 step forward. You see a red key 1 step left
and 3 steps forward. You see a yellow ball 1 step left. You see a blue box 1 step right and 3 steps forward. You 
see a blue ball 2 steps right and 2 steps forward. You see a yellow ball 2 steps right
On your turn, simply type your message.
The game lasts for 20 turns in total.
<|im_end|>

In [ ]:
# Show 5 color-coded *trajectories* (user -> assistant(tool_call) -> tool -> ...)
from act_prm.environments import get_env

babyai_env = get_env(
    name="babyai_text",
    env_name="BabyAI-MixedTestLocal-v0",
    num_train_samples=20,
    num_val_samples=0,
    num_test_samples=0,
    max_turns=20,
    seed=0,
)

for sample_idx in range(5):
    state = babyai_env.reset(sample_idx=sample_idx)
    system_message = {"role": "system", "content": state.system_prompt}

    msg_text = hf_tokenizer.apply_chat_template(
        [system_message] + state.action_trajectory,
        tokenize=False,
        tools=state.tools,
    )

    print("\n" + "=" * 120)
    print(f"TRAJECTORY EXAMPLE {sample_idx}")
    rich_print_messages(msg_text)

Failed to build gold path: module 'numpy' has no attribute 'bool8'



TRAJECTORY EXAMPLE 0


<|im_start|>system
You are a helpful assistant that can answer questions and call tools.

You are an agent playing BabyAI-Text, a grid-based game with text observations.
Your goal is to follow the mission and reach the win condition.

## Rules:
- You must call exactly one tool per turn to choose an action.
- Only the listed tools are valid actions.
- You have 20 steps to complete the task. Restarting is forbidden.

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "name": "turn_left", "description": "Rotate the agent 90 degrees to the left.", "parameters": 
{"type": "object", "properties": {}}}
{"type": "function", "name": "turn_right", "description": "Rotate the agent 90 degrees to the right.", 
"parameters": {"type": "object", "properties": {}}}
{"type": "function", "name": "go_forward", "description": "Move forward one cell.", "parameters": {"type": 
"object", "properties": {}}}
{"type": "function", "name": "pick_up", "description": "Pick up the object in front of the agent.", "parameters": 
{"type": "object", "properties": {}}}
{"type": "function", "name": "drop", "description": "Drop the object the agent is carrying.", "parameters": 
{"type": "object", "properties": {}}}
{"type": "function", "name": "toggle", "description": "Toggle (use) the object in front of the agent.", 
"parameters": {"type": "object", "properties": {}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML 
tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call><|im_end|>
<|im_start|>user
You are playing 'BabyAI-Text'.
Your goal is to open the red door.
Available actions are turn left, turn right, go forward, pick up, drop, toggle.
You see a wall 4 steps forward. You see a yellow key 2 steps left and 1 step forward. You see a red key 1 step left
and 3 steps forward. You see a yellow ball 1 step left. You see a blue box 1 step right and 3 steps forward. You 
see a blue ball 2 steps right and 2 steps forward. You see a yellow ball 2 steps right
On your turn, simply type your message.
The game lasts for 20 turns in total.
<|im_end|>

Failed to build gold path: module 'numpy' has no attribute 'bool8'



TRAJECTORY EXAMPLE 1


...


TRAJECTORY EXAMPLE 3


<|im_start|>system
You are a helpful assistant that can answer questions and call tools.

You are an agent playing BabyAI-Text, a grid-based game with text observations.
Your goal is to follow the mission and reach the win condition.

## Rules:
- You must call exactly one tool per turn to choose an action.
- Only the listed tools are valid actions.
- You have 20 steps to complete the task. Restarting is forbidden.

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "name": "turn_left", "description": "Rotate the agent 90 degrees to the left.", "parameters": 
{"type": "object", "properties": {}}}
{"type": "function", "name": "turn_right", "description": "Rotate the agent 90 degrees to the right.", 
"parameters": {"type": "object", "properties": {}}}
{"type": "function", "name": "go_forward", "description": "Move forward one cell.", "parameters": {"type": 
"object", "properties": {}}}
{"type": "function", "name": "pick_up", "description": "Pick up the object in front of the agent.", "parameters": 
{"type": "object", "properties": {}}}
{"type": "function", "name": "drop", "description": "Drop the object the agent is carrying.", "parameters": 
{"type": "object", "properties": {}}}
{"type": "function", "name": "toggle", "description": "Toggle (use) the object in front of the agent.", 
"parameters": {"type": "object", "properties": {}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML 
tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call><|im_end|>
<|im_start|>user
You are playing 'BabyAI-Text'.
Your goal is to go to the yellow box.
Available actions are turn left, turn right, go forward, pick up, drop, toggle.
You see a wall 2 steps right. You see a red key 3 steps left and 2 steps forward. You see a red key 3 steps left. 
You see a grey ball 3 steps forward
On your turn, simply type your message.
The game lasts for 20 turns in total.
<|im_end|>

Failed to build gold path: module 'numpy' has no attribute 'bool8'



TRAJECTORY EXAMPLE 4


<|im_start|>system
You are a helpful assistant that can answer questions and call tools.

You are an agent playing BabyAI-Text, a grid-based game with text observations.
Your goal is to follow the mission and reach the win condition.

## Rules:
- You must call exactly one tool per turn to choose an action.
- Only the listed tools are valid actions.
- You have 20 steps to complete the task. Restarting is forbidden.

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "name": "turn_left", "description": "Rotate the agent 90 degrees to the left.", "parameters": 
{"type": "object", "properties": {}}}
{"type": "function", "name": "turn_right", "description": "Rotate the agent 90 degrees to the right.", 
"parameters": {"type": "object", "properties": {}}}
{"type": "function", "name": "go_forward", "description": "Move forward one cell.", "parameters": {"type": 
"object", "properties": {}}}
{"type": "function", "name": "pick_up", "description": "Pick up the object in front of the agent.", "parameters": 
{"type": "object", "properties": {}}}
{"type": "function", "name": "drop", "description": "Drop the object the agent is carrying.", "parameters": 
{"type": "object", "properties": {}}}
{"type": "function", "name": "toggle", "description": "Toggle (use) the object in front of the agent.", 
"parameters": {"type": "object", "properties": {}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML 
tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call><|im_end|>
<|im_start|>user
You are playing 'BabyAI-Text'.
Your goal is to go to the red key.
Available actions are turn left, turn right, go forward, pick up, drop, toggle.
You see a wall 1 step right. You see a green ball 3 steps left. You see a yellow ball 1 step left and 2 steps 
forward. You see a purple box 2 steps forward
On your turn, simply type your message.
The game lasts for 20 turns in total.
<|im_end|>

In [ ]:
import numpy as np
print(np.__version__)
print(hasattr(np, "bool8"))

1.26.4
True


/var/folders/mf/96xkkw8j0992d2xk2qf_h3kc0000gn/T/ipykernel_74420/1423151961.py:3: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  print(hasattr(np, "bool8"))
